In [1]:
import numpy as np
import itertools

In [2]:
M = EuclideanSpace(3, 'M', start_index=1)
X.<x,y,z> = M.chart()
# e = M.vector_frame('e')
# de = M.coframes()[1] 

eX = X.frame()
dX = X.coframe()



In [3]:
# Declaration of 0-form (scalar field)
f = M.scalar_field(x^2)
f.display()

# Evaluation of 0-form
# p = M.point((1,2,3))
# f(p)


M → ℝ
(x, y, z) ↦ x^2

In [31]:
# Declaration 1-form
a = M.diff_form(1, {eX:[x*y*z, x*y*z, x*y*z]}, name='a')
a.display(X)
# a[X, :] = [x*y*z, 0, x*y*z]
# a.display(eX)

# evaluation 1-form
# a(eX[1])(p)

a = x*y*z dx + x*y*z dy + x*y*z dz

In [5]:
a.hodge_dual().comp(X.frame())[:, X]

[     0      0      0]
[     0      0  x*y*z]
[     0 -x*y*z      0]

In [6]:
# Declaration 2-form
b = M.diff_form(2, name='b')
b[eX, 1, 2] = x*y*z #dx^dy
b[eX, 1, 3] = x*y^2 #dx^dz
b[eX, 2, 3] = z*x^2*y^3-2*x*y #dy^dz
b.display(X)


# evaluation 1-form
# p = M.point((1,2,3))
# b(eX[1], eX[2])(p)

b = x*y*z dx∧dy + x*y^2 dx∧dz + (x^2*y^3*z - 2*x*y) dy∧dz

In [93]:
numerical_sampler_kform(M, a, (1,2,3))

array([6., 0., 0.])

In [7]:
def numerical_sampler_kform(M, k_form, point):
    """
    Sample numerical values of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the k-form at the point as a tensor
    """
    
    # Check if the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        p = M.point(point)
        return np.array(float(k_form(p)))
    
    if k_form.degree() == 1:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([float(k_form(eX[i])(p)) for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 2:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[float(k_form(eX[i], eX[j])(p)) for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])
    
    if k_form.degree() == 3:
        p = M.point(point)
        X.<x,y,z> = M.chart()
        eX = X.frame()
        return np.array([[[float(k_form(eX[i], eX[j], eX[k])(p)) for k in range(1,M.dimension()+1)] for j in range(1,M.dimension()+1)] for i in range(1,M.dimension()+1)])

In [8]:
def numerical_exterior_derivative(M, k_form, point, epsilon=1e-5):
    """
    Sample numerical values of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: numerical values of the exterior derivative of the k-form at the point as a tensor
    """
    
    # If the k-form is a 0-form (scalar field)
    if k_form.degree() == 0: 
        dw = np.zeros(3)
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            dw[i] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point + epsilon*directions[i]) - numerical_sampler_kform(M, k_form, point - epsilon*directions[i]))
        
        return dw
    
    # If the k-form is a 1-form
    elif k_form.degree() == 1:
        dw = np.zeros((3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                if i!=j:
                    dw[i,j] = (1/(2*epsilon))*(numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i] +\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j] -\
                                            numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i] -\
                                            numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j])
        
        return dw
    
    elif k_form.degree() == 2:
        dw = np.zeros((3,3,3))
        directions = [np.array(np.eye(3, dtype=int)[i]) for i in range(3)]
        point = np.array(point)
        for i in range(3):
            for j in range(3):
                for k in range(3):
                    if i!=j and j!=k and i!=k:
                        dw[i,j,k] = (1/(2*epsilon))*(+numerical_sampler_kform(M, k_form, point +  epsilon*directions[i])[j,k] \
                                            -numerical_sampler_kform(M, k_form, point -  epsilon*directions[i])[j,k] \
                                            -numerical_sampler_kform(M, k_form, point +  epsilon*directions[j])[i,k] \
                                            +numerical_sampler_kform(M, k_form, point -  epsilon*directions[j])[i,k] \
                                            +numerical_sampler_kform(M, k_form, point +  epsilon*directions[k])[i,j] \
                                            -numerical_sampler_kform(M, k_form, point -  epsilon*directions[k])[i,j])
                                            
        return dw
    
    elif k_form.degree() >= 3:
        assert False, "The exterior derivative of a 3-form is a 4-form, which is not defined in 3D space."
        
        
        

In [9]:
def count_inversions(arr):
    """Count inversions in O(n log n) time via merge sort"""
    def merge_sort(lst):
        if len(lst) <= 1:
            return lst, 0
        mid = len(lst) // 2
        left, inv_l = merge_sort(lst[:mid])
        right, inv_r = merge_sort(lst[mid:])
        merged = []
        i = j = inv_m = 0
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                merged.append(left[i])
                i += 1
            else:
                merged.append(right[j])
                inv_m += len(left) - i  # key line
                j += 1
        merged += left[i:]
        merged += right[j:]
        return merged, inv_l + inv_r + inv_m
    _, count = merge_sort(arr)
    return count

def perm_sign(p):
    """Fast sign using inversion count"""
    return 1 if count_inversions(p) % 2 == 0 else -1

In [10]:
import itertools

for p in itertools.permutations([0, 1, 2]):
    sign = perm_sign(p)
    print(f"{p} -> sign: {sign}")

(0, 1, 2) -> sign: 1
(0, 2, 1) -> sign: -1
(1, 0, 2) -> sign: -1
(1, 2, 0) -> sign: 1
(2, 0, 1) -> sign: 1
(2, 1, 0) -> sign: -1


In [11]:
list(itertools.combinations([0, 1, 2], 2))

[(0, 1), (0, 2), (1, 2)]

In [12]:
np.zeros((3,3)).transpose()

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [38]:
def numerical_Hodge_dual(M, k_form, point):
    # TODO: replace  numerical_sampler_kform(M, k_form, point) by a tensor (3,3,...)
    if k_form.degree() == 0: 
        star = np.zeros((3,3,3))
        perms = list(itertools.permutations([0, 1, 2]))
        for p in perms:
            sign = perm_sign(p)
            star[p[0], p[1], p[2]] = sign * numerical_sampler_kform(M, k_form, point)
        return star
    
    elif k_form.degree() == 1:
        star = np.zeros((3,3))
        perms = list(itertools.combinations([0, 1, 2], 2))
        
        for p in perms:
            star[p[0], p[1]] = ((-1)**(1+p[0]+p[1]))*numerical_sampler_kform(M, k_form, point)[list(set([0,1,2]) - set([p[0],p[1]]))[0]]
        star = star - star.transpose()
        return star
    
    elif k_form.degree() == 2:
        star = np.zeros(3)
        
        for p in range(3):
            star[p] = ((-1)**p)*(numerical_sampler_kform(M, k_form, point)[list(set([0,1,2]) - set([p]))[0], list(set([0,1,2]) - set([p]))[1]])
        return star
    
    elif k_form.degree() == 3:
        star = numerical_sampler_kform(M, k_form, point)[0,1,2]
        return np.array(star)
    
    else:
        assert False, "Not defined."
        
        
        
        

In [37]:
for p in list(itertools.combinations([0, 1, 2], 2)):
    print(p[0])

0
0
1


In [14]:
c = M.diff_form(0, name='c')
# c[eX, 1, 2, 3] = x*y*z #dx^dy^dz
# c.display(X)

In [40]:
a.hodge_dual().display(X)

*a = x*y*z dx∧dy - x*y*z dx∧dz + x*y*z dy∧dz

In [29]:
(a.exterior_derivative().hodge_dual().exterior_derivative().hodge_dual()).display(X)

*d*da = z dy + y dz

In [41]:
numerical_Hodge_dual(M, a, (-1,2,3))

array([[ 0., -6.,  6.],
       [ 6.,  0., -6.],
       [-6.,  6.,  0.]])

In [18]:
a.exterior_derivative().comp(X.frame())[:, X]

[   0 -x*z -x*y]
[ x*z    0    0]
[ x*y    0    0]

In [22]:
numerical_exterior_derivative(M, a, (1,2,3)) 

array([[ 0., -3., -2.],
       [ 3.,  0.,  0.],
       [ 2.,  0.,  0.]])

In [12]:
k_form = b
np.array(k_form.exterior_derivative().comp(X.frame())[:, X]).flatten()   

array([0, 0, 0, 0, 0, 2*x*y^3*z - (x + 2)*y, 0, -2*x*y^3*z + (x + 2)*y, 0,
       0, 0, -2*x*y^3*z + (x + 2)*y, 0, 0, 0, 2*x*y^3*z - (x + 2)*y, 0, 0,
       0, 2*x*y^3*z - (x + 2)*y, 0, -2*x*y^3*z + (x + 2)*y, 0, 0, 0, 0, 0],
      dtype=object)

In [13]:
def test_numerical_approximation(M, k_form, point, epsilon=1e-3):
    """
    Test the numerical approximation of the exterior derivative of a k-form at a point
    :param M: manifold
    :param k_form: k-form to sample
    :param point: point at which to sample
    :return: True if the numerical approximation is close to the analytical value, False otherwise
    """
    # def eval_symbolic_array(exprs, point):
    #     values = {x: point[0], y: point[1], z: point[2]}
    #     return np.array([float(SR(str(e)).substitute(values)) for e in exprs])

    
    # Compute the numerical exterior derivative
    numerical_derivative = np.array(numerical_exterior_derivative(M, k_form, point, epsilon=epsilon)).flatten()
    
    # X.<x,y,z>= M.chart()
    # analytic_coomp = np.array(k_form.exterior_derivative().comp(X.frame())[:, X]).flatten()     
    # analytical_derivative = eval_symbolic_array(analytic_coomp, point)
    analytical_derivative = np.array(numerical_sampler_kform(M, k_form.exterior_derivative(), point)).flatten()
    
    print("Flatten numerical derivative: ", numerical_derivative)
    print("Flatten analytical derivative: ", analytical_derivative)
    
    return np.allclose(numerical_derivative, analytical_derivative, atol=epsilon)     
        

In [14]:
numerical_exterior_derivative(M, b, (1,2,3))

array([[[  0.,   0.,   0.],
        [  0.,   0.,  42.],
        [  0., -42.,   0.]],

       [[  0.,   0., -42.],
        [  0.,   0.,   0.],
        [ 42.,   0.,   0.]],

       [[  0.,  42.,   0.],
        [-42.,   0.,   0.],
        [  0.,   0.,   0.]]])

In [15]:
b.exterior_derivative().display(X)

db = (2*x*y^3*z - (x + 2)*y) dx∧dy∧dz

In [16]:
np.array(numerical_sampler_kform(M, b.exterior_derivative(), (1,2,3))).flatten()

array([  0.,   0.,   0.,   0.,   0.,  42.,   0., -42.,   0.,   0.,   0.,
       -42.,   0.,   0.,   0.,  42.,   0.,   0.,   0.,  42.,   0., -42.,
         0.,   0.,   0.,   0.,   0.])

In [17]:
test_numerical_approximation(M, b, (32,1,986))

Flatten numerical derivative:  [     0.              0.              0.              0.
      0.          63070.00000007      0.         -63070.00000007
      0.              0.              0.         -63070.00000006
      0.              0.              0.          63070.00000007
      0.              0.              0.          63070.00000008
      0.         -63070.00000007      0.              0.
      0.              0.              0.        ]
Flatten analytical derivative:  [     0.      0.      0.      0.      0.  63070.      0. -63070.      0.
      0.      0. -63070.      0.      0.      0.  63070.      0.      0.
      0.  63070.      0. -63070.      0.      0.      0.      0.      0.]


True